# 2019-2021 CAO data analysis
***

1. Save original data file
2. Load CAO data (2019-2021) into spreadsheet
    - 2021: from website to csv file
    - 2020: from excel to csv file
    - 2019: from pdf to csv file
3. Clean data, ensuring data integrity 
4. Analysis
    - plots of overview by year
    - overall points (sum/average) + plots
    - courses that has raised points by year
    - 


In [28]:
# using Anaconda to install Camelot
#$ conda install -c conda-forge camelot-py

In [1]:
# request data from website for 2021 data
import requests as rq

# Data was in a particular pattern, extract using regular expression
import re 

# date time for file naming
import datetime as dt

# data frame
import pandas as pd

# downloading 2020 data
import urllib.request as urlrq

# convert pdf to csv/df for 2019 data
import camelot as cam 

# for filename matching for 2019 data
import glob

In [2]:
# get current date and time, for file path creation later
now = dt.datetime.now()

#format as a string
nowstr = now.strftime("%Y%m%d_%H%M%S")

## 2021 CAO data 
link to CAO 2021 data website : http://www2.cao.ie/points/l8.php <br/>
***
*Steps to turn data from online source to data frame:*
1. Get data from online source, then save as html with correct encoding
2. Extract relevant data using Regular Expression
3. Save extracted data as csv file then read in as data frame

Using Regular Expression to search and extract data, converting raw HTML to csv

In [31]:
# Get data from CAO website to notebook
points = rq.get("http://www2.cao.ie/points/l8.php")
points
# <Response [200]> means successfully getting the data

<Response [200]>

**Saving original data set** <br>
Using datatime string created earlier to name and identify files saved at different point of time

In [32]:
# create file path with date time to save the data 
pathHTML = "caoData2021_"+nowstr+".html"

In [33]:
# loop through the lines to have a look of the content
for line in points.iter_lines():
    print (line)

b'<html>'
b'<BODY>'
b'<a name="deg"><hr>'
b'<h3><center> Points Required for Entry to 2021 Level 8 Courses</h3></center><p>'
b'<A HREF="l76.php">Go to Level 7/6 List of Institutions.</a><p>'
b'<PRE><b>'
b'*     Not all on this points score were offered places.'
b'#     Test / Interview / Portfolio / Audition'
b'AQA   All qualified applicants'
b'v     New competition for available places'
b' '
b'</PRE>'
b'<dl>'
b'<p>'
b'<dt><a href="#ac8"><b>American College Dublin</b></a>'
b'<dt><a href="#al8"><b>Athlone Institute of Technology</b></a>'
b'<dt><a href="#by8"><b>IBAT College Dublin</b></a>'
b'<dt><a href="#cm8"><b>Marino Institute of Education</b></a>'
b'<dt><a href="#pc8"><b>Carlow College</b></a>'
b'<dt><a href="#ct8"><b>CCT College Dublin</b></a>'
b'<dt><a href="#cr8"><b>Cork Institute of Technology</b></a>'
b'<dt><a href="#db8"><b>Dublin Business School</b></a>'
b'<dt><a href="#dc8"><b>Dublin City University</b></a>'
b'<dt><a href="#dk8"><b>Dundalk Institute of Technology</b></a>'
b'

Before digging into extracting relevant data, we have to fix the decoding error first. <br/>
After some explorations, there is an error in decoding text. The server uses iso-8859-1 however it does not decode \x96, therefore we use standard cp1252 which is very similar but will decode \x96.

In [34]:
# change encoding to "cp1252"
original_encoding = points.encoding
points.encoding = 'cp1252'

In [35]:
# save the original html file
with open(pathHTML, "w") as f:
    f.write(points.text)

**Extract relevant data** <br>
We will need to get rid of the html tags and extract only the course code, course name, points for round 1 and 2.

There seems to be a pattern to the data that we need, Using Regular Expression (Regex) to match the pattern.

In [36]:
# pattern of the data we want to extract
# course code consisting of 2 uppercase letters and 3 integers
regex = r'([A-Z]{2}[0-9]{3})(.*)'
# compile the regex to match the lines
re_course = re.compile(regex)

We will now move on to iterating all the lines in the html, with the correct encoding and extracting relevant data using regular expression. The extracted data will then be "split" and "join" into csv ready format and saved in a csv file. 

In [37]:
# Save the file into csv using the path pattern
path2021 = "caoData2021_csv_" + nowstr + ".csv"

# Keep track of courses processed
no_lines = 0

# Open the csv file for writing
with open (path2021,"w") as f:
    # write head row, string join method, with commas separting the columns for csv
    f.write(','.join(['code','title','pointsR1','pointsR2'])+'\n')
    #loop through the lines
    for line in points.iter_lines():
        #decode the line using wrong encoding
        dline = line.decode("cp1252")
        #match lines that represent courses 
        if re_course.fullmatch(dline):
            #add 1 to line counter
            no_lines = no_lines + 1
            # The course code.
            course_code = dline[:5]
            # The course title.
            course_title = dline[7:57].strip()
            # Round one points.
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print total number of processes lines
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


In [38]:
df2021 = pd.read_csv(path2021, encoding="cp1252")

In [39]:
df2021

# there's a lot of "NaN" in pointsR2, 
# because not all courses went through R2
# might worth looking into number and courses that went through R2

,code,title,pointsR1,pointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


## 2020 data - save original and load into csv file

In [40]:
url2020 = "http://www2.cao.ie/points/CAOPointsCharts2020.xlsx"

#### Save original File

In [41]:
# create file path
pathxlsx = 'cao2020_' + nowstr + '.xlsx'

In [42]:
urlrq.urlretrieve(url2020, pathxlsx)

('cao2020_20211126_062621.xlsx', <http.client.HTTPMessage at 0x21b579bee50>)

#### Load into pandas

In [43]:
# first 10 rows are preemble, don't need them in the data frame
df2020 = pd.read_excel(url2020, skiprows=10)
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# spot check
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [45]:
# create a file path for pandas data
path2020 = "cao2020_" + nowstr + ".csv"

In [46]:
# save it.
df2020.to_csv(path2020)

## 2019 data - save original and load into csv file

reference: <br>
https://www.analyticsvidhya.com/blog/2020/08/how-to-extract-tabular-data-from-pdf-document-using-camelot-in-python/
https://camelot-py.readthedocs.io/en/master/user/quickstart.html


**Steps to turn data from pdf file to data frame**
1. 

In [4]:
# read in 2019 data from pdf using camelot
table2019 = cam.read_pdf("CAO data/caoData2019_original.pdf", pages='all', flavor='stream')

In [5]:
# To see how many tables in the pdf
table2019
# camelot separted the table into 18 tables, as there were 18 pages

<TableList n=18>

In [6]:
# parsing report shows the accuracy of extraction
for i in range(18):
    print(table2019[i].parsing_report)

# data were most likely extracted correctly

{'accuracy': 99.58, 'whitespace': 6.91, 'order': 1, 'page': 1}
{'accuracy': 100.0, 'whitespace': 4.02, 'order': 1, 'page': 2}
{'accuracy': 100.0, 'whitespace': 3.57, 'order': 1, 'page': 3}
{'accuracy': 100.0, 'whitespace': 6.7, 'order': 1, 'page': 4}
{'accuracy': 100.0, 'whitespace': 3.12, 'order': 1, 'page': 5}
{'accuracy': 100.0, 'whitespace': 2.68, 'order': 1, 'page': 6}
{'accuracy': 100.0, 'whitespace': 4.46, 'order': 1, 'page': 7}
{'accuracy': 100.0, 'whitespace': 7.59, 'order': 1, 'page': 8}
{'accuracy': 97.37, 'whitespace': 10.71, 'order': 1, 'page': 9}
{'accuracy': 100.0, 'whitespace': 3.57, 'order': 1, 'page': 10}
{'accuracy': 100.0, 'whitespace': 4.02, 'order': 1, 'page': 11}
{'accuracy': 100.0, 'whitespace': 2.68, 'order': 1, 'page': 12}
{'accuracy': 100.0, 'whitespace': 2.68, 'order': 1, 'page': 13}
{'accuracy': 100.0, 'whitespace': 5.36, 'order': 1, 'page': 14}
{'accuracy': 100.0, 'whitespace': 2.68, 'order': 1, 'page': 15}
{'accuracy': 99.92, 'whitespace': 6.7, 'order': 1

In [18]:
# export the tables to 18 csv files inside a zip file (compressed) 
#table2019.export(CAO2019.csv", f="csv", compress=True)

In [37]:
import os ### not finished
#os.chdir("CAO2019")
extension="csv"
all_filenames = []
path="CAO2019"
filenames = [glob.glob('*.{}'.format(extension))]

# a list of all csv files, but sorted in wrong order
for i in filenames:
    all_filenames=[i]
    #print(all_filenames)
    
# combine all files in the list
for f in all_filenames:
    combined_csv=pd.concat(pd.read_csv(f))
    #export to csv
    combined_csv.to_csv("2019_csv.csv", index=False)
    
# after combine to 1 csv, turn it into df

ValueError: Invalid file path or buffer object type: <class 'list'>

In [45]:
# to read all the csv files and concat into 1 data frame
# https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

# get data file names
path ='CAO2019' #the unzipped folder with the 18 csv files
filenames = glob.glob(path + "/*.csv")

#dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# Concatenate all data into one DataFrame
df2019 = pd.concat(dfs)

'''df=[]
df.append(pd.read_csv(path+"\CAO2019-page-9-table-1.csv"))'''

In [48]:
df2019
# now the columns seems messed up, what about join all the csv into 1 first?

,*,Not all on this points score were offered places,Unnamed: 2,Unnamed: 3,Unnamed: 0,2019 Level 8 EOS
0,#,Test / Interview / Portfolio / Audition,NaN,NaN,NaN,NaN
1,AQA,All qualified applicants,NaN,NaN,NaN,NaN
2,Course Code INSTITUTION and COURSE,NaN,EOS,Mid,NaN,NaN
3,NaN,Athlone Institute of Technology,NaN,NaN,NaN,NaN
4,AL801,Software Design with Virtual Reality and Gaming,304,328,NaN,NaN
...,...,...,...,...,...,...
50,NaN,NaN,487*,520,TR032,Engineering
51,NaN,NaN,465*,488,TR033,Computer Science
52,NaN,NaN,589*,602,TR034,Management Science and Information Systems Stu...
53,NaN,NaN,554,601,TR035,Theoretical Physics


In [32]:
# have a look of the df
df2019[:50] #row0-3 are preembles

,*,Not all on this points score were offered places,Unnamed: 2,Unnamed: 3,Unnamed: 0,2019 Level 8 EOS
0,#,Test / Interview / Portfolio / Audition,NaN,NaN,NaN,NaN
1,AQA,All qualified applicants,NaN,NaN,NaN,NaN
2,Course Code INSTITUTION and COURSE,NaN,EOS,Mid,NaN,NaN
3,NaN,Athlone Institute of Technology,NaN,NaN,NaN,NaN
4,AL801,Software Design with Virtual Reality and Gaming,304,328,NaN,NaN
5,AL802,Software Design with Cloud Computing,301,306,NaN,NaN
6,AL803,Software Design with Mobile Apps and Connected...,309,337,NaN,NaN
7,AL805,Network Management and Cloud Infrastructure,329,442,NaN,NaN
8,AL810,Quantity Surveying,307,349,NaN,NaN
9,AL820,Mechanical and Polymer Engineering,300,358,NaN,NaN


In [27]:
df2019[-30:-1]

,*,Not all on this points score were offered places,Unnamed: 2,Unnamed: 3,Unnamed: 0,2019 Level 8 EOS
938,NaN,NaN,555,566,RC005,Pharmacy
939,NaN,NaN,#60,62,RC101,Medicine - Graduate Entry (GAMSAT required)
940,NaN,NaN,NaN,NaN,NaN,Trinity College Dublin
941,NaN,NaN,#414,473,TR002,Music
942,NaN,NaN,486,518,TR003,History
943,NaN,NaN,532,552,TR004,Law
944,NaN,NaN,391,446,TR005,Philosophy
945,NaN,NaN,555*,577,TR006,Psychology
946,NaN,NaN,499,529,TR007,Clinical Speech and Language Studies
947,NaN,NaN,#430,465,TR009,Music Education


### Concat and join data of 2020 and 2021
https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html
join them by course code and course title, drop the duplicated rows, then join the points by code and title

In [55]:
courses2021 = df2021[["code", "title"]]
courses2021

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
944,WD211,Creative Computing
945,WD212,Recreation and Sport Management
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [57]:
courses2020 = df2020[["COURSE CODE2","COURSE TITLE"]]
#change the name of columns to match the one in 2021
courses2020.columns=["code","title"]
courses2020

,code,title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1459,WD208,Manufacturing Engineering
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


In [76]:
allcourses = pd.concat([courses2021, courses2020], ignore_index=True)
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2408,WD208,Manufacturing Engineering
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [77]:
allcourses.iloc[175]

code                      AC120
title    International Business
Name: 175, dtype: object

In [78]:
allcourses.iloc[949]

code                      AC120
title    International Business
Name: 949, dtype: object

<br>
After concatenating, there will be duplicated rows from data of 2 different years. In order to remove duplicates, match the courses by course code as course code should be unique to a particular course in a particular HEI.
<br>

In [79]:
allcourses.sort_values("code")

,code,title
175,AC120,International Business
949,AC120,International Business
950,AC137,Liberal Arts
176,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
...,...,...
2411,WD212,Recreation and Sport Management
2412,WD230,Mechanical and Manufacturing Engineering
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [80]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated()]

,code,title
949,AC120,International Business
950,AC137,Liberal Arts
952,AD102,Graphic Design and Moving Image Design (portfo...
955,AD204,Fine Art (portfolio)
956,AD211,Fashion Design (portfolio)
...,...,...
2404,WD200,Arts (options)
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [81]:
# Returns a copy of the data frame with only distinct rows.
allcourses.drop_duplicates()

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2403,WD195,Architectural and Building Information Modelli...
2405,WD205,Molecular Biology with Biopharmaceutical Science
2406,WD206,Electronic Engineering
2407,WD207,Mechanical Engineering


In [70]:
# these are the extra copies of duplicated rows
allcourses[allcourses.duplicated(subset=["code"])]

,code,title
949,AC120,International Business
950,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
952,AD102,Graphic Design and Moving Image Design (portfo...
953,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
2404,WD200,Arts (options)
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [82]:
# Returns a data frame with only distinct course code.
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)

In [83]:
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
1512,WD188,Applied Health Care
1513,WD205,Molecular Biology with Biopharmaceutical Science
1514,WD206,Electronic Engineering
1515,WD207,Mechanical Engineering


In [84]:
# set course code as index (2021 data)
df2021.set_index('code', inplace=True)
df2021.columns = ['title', 'points_r1_2021', 'points_r2_2021']
df2021

,title,points_r1_2021,points_r2_2021
code,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN
AL802,Software Design in Artificial Intelligence for...,313,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,NaN
AL805,Computer Engineering for Network Infrastructure,321,NaN
AL810,Quantity Surveying,328,NaN
...,...,...,...
WD211,Creative Computing,270,NaN
WD212,Recreation and Sport Management,262,NaN
WD230,Mechanical and Manufacturing Engineering,230,230


In [85]:
# Set course code as index(2021 & 2020).
allcourses.set_index('code', inplace=True)

In [86]:
allcourses = allcourses.join(df2021[['points_r1_2021']])
allcourses

,title,points_r1_2021
code,,
AL801,Software Design for Virtual Reality and Gaming,300
AL802,Software Design in Artificial Intelligence for...,313
AL803,Software Design for Mobile Apps and Connected ...,350
AL805,Computer Engineering for Network Infrastructure,321
AL810,Quantity Surveying,328
...,...,...
WD188,Applied Health Care,NaN
WD205,Molecular Biology with Biopharmaceutical Science,NaN
WD206,Electronic Engineering,NaN


In [87]:
# change the names of column in df2020
df2020_r1 = df2020[['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns = ['code', 'points_r1_2020']
df2020_r1

,code,points_r1_2020
0,AC120,209
1,AC137,252
2,AD101,#+matric
3,AD102,#+matric
4,AD103,#+matric
...,...,...
1459,WD208,188
1460,WD210,279
1461,WD211,271
1462,WD212,270


In [88]:
# set course code as index (2020).
df2020_r1.set_index('code', inplace=True)
df2020_r1

,points_r1_2020
code,
AC120,209
AC137,252
AD101,#+matric
AD102,#+matric
AD103,#+matric
...,...
WD208,188
WD210,279
WD211,271


In [89]:
# Join 2020 points to allcourses.
allcourses = allcourses.join(df2020_r1)
allcourses

,title,points_r1_2021,points_r1_2020
code,,,
AL801,Software Design for Virtual Reality and Gaming,300,303
AL802,Software Design in Artificial Intelligence for...,313,332
AL803,Software Design for Mobile Apps and Connected ...,350,337
AL805,Computer Engineering for Network Infrastructure,321,333
AL810,Quantity Surveying,328,319
...,...,...,...
WD188,Applied Health Care,NaN,201
WD205,Molecular Biology with Biopharmaceutical Science,NaN,228
WD206,Electronic Engineering,NaN,179


### Concat and join data of 2019 and 2020



## Analysis
***

Describe and summary plots of 3 years of data
- max, min, mean, std points
- course with highest and lowest points
- 

In [99]:
allcourses.describe(include="all") 

,title,points_r1_2021,points_r1_2020
count,1517,928,1437
unique,1004,377,389
top,Business,300,300
freq,31,18,26


Comparing data of 2019,2020, 2021 using pandas functions

Create plots and visualizations

References:
[1]
[2]